In [1]:
# k crossover d，close > ema 8, ema 8 > ema 18, ema 18 > 38, take profit atr, stop loss atr
# Variables :
# time - 15m, 1h
# start time - 1609459200000, 1641028611000, 1672564611000, 1654045261000, 1651366861000 (2021, 2022, 2023)
# start_time = 1635743450000 2021/11 end_time = 1654060250000 2022/6 BEAR

In [2]:
# ! conda install -c conda-forge ta --yes

In [3]:
import requests
import pandas as pd
import ta
import datetime as dt
import numpy as np
import time
import itertools
import os

In [4]:
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

In [5]:
# start_time_arr = [1672531200000, 1651366861000]

interval_arr = ['1h', '15m']

# how kd is crossed
kd_dir_arr = ['Any', 'Up']

sl_atr_arr = [1, 3, 5, 7]

tp_atr_arr = [3, 5, 9, 12]

# stop loss determine candlestick
sl_det_arr = ['Close']

In [6]:
timezone = 8
# symbol = 'nearusdt'
symbol_arr = ['ethusdt', 'ftmusdt', 'sandusdt', 'woousdt', 'dydxusdt']
start_time_d = 1635743450000
end_time = round(time.time() * 1000)
# end_time = 1654060250000 
# step between timestamps in milliseconds
step = 1000

In [7]:
dataframes = {}
def create_raw(symbol_arr, interval_arr, start_time, end_time, step):
        
    for interval in interval_arr:
        for symbol in symbol_arr:
            # Fetch the data using batch requests
            data = []    
            while start_time < end_time:
                limit = min(step, end_time - start_time + 1)  # Adjust the limit for the last batch
                response = client.get_klines(symbol=symbol.upper(), interval=interval, limit=limit, startTime=start_time)

                if len(response) == 0:
                    break  # No more data available, exit the loop
                data.extend(response)
                start_time = response[-1][0] + 1

            # Convert the data to a DataFrame
            columns = [
                "Open_Time", "Open", "High", "Low", "Close", "Volume", "Close_Time",
                "Quote asset volume", "Number of trades", "Taker buy base asset volume",
                "Taker buy quote asset volume", "Ignore"
            ]
            raw_df = pd.DataFrame(data, columns=columns)   

            raw_df = raw_df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']]

            dataframes[f'df_{interval}_{symbol}'] = raw_df
            start_time = start_time_d

In [8]:
loop_start_time = time.time()
create_raw(symbol_arr, interval_arr, start_time_d, end_time, step)
loop_end_time = time.time()
print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")
print(dataframes)

Time taken to execute for loop: 29.867750644683838 seconds
{'df_1h_ethusdt':            Open_Time           Open          Close           High  \
0      1635746400000  4225.00000000  4241.99000000  4259.14000000   
1      1635750000000  4241.99000000  4302.93000000  4320.00000000   
2      1635753600000  4302.94000000  4311.11000000  4338.07000000   
3      1635757200000  4311.11000000  4316.56000000  4344.41000000   
4      1635760800000  4316.55000000  4328.94000000  4335.00000000   
...              ...            ...            ...            ...   
13935  1685916000000  1902.01000000  1901.61000000  1912.89000000   
13936  1685919600000  1901.61000000  1890.01000000  1904.37000000   
13937  1685923200000  1890.00000000  1882.12000000  1890.63000000   
13938  1685926800000  1882.12000000  1884.25000000  1885.00000000   
13939  1685930400000  1884.25000000  1883.15000000  1887.00000000   

                 Low          Volume  
0      4198.55000000  13908.93710000  
1      4241.4600

In [9]:
def get_klines(symbol, interval, start_time, end_time):
    global dataframes
    df = dataframes[f'df_{interval}_{symbol}']
    df = df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)
    df = df.set_index('Open_Time')

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
    df = df[~df.index.duplicated(keep='first')]

    return df

In [10]:
def check_cross(df, kd_dir):
    up = df['slow_k'] > df['slow_d']
    down = df['slow_k'] < df['slow_d']
    if kd_dir == 'Up':
        return up.diff() & up
    if kd_dir == 'Any':
        return up.diff()
    if kd_dir == 'Down':
        return down.diff() & down


def indicators(df, kd_dir):

# ema
    for i in (8, 18, 38):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)
    
# rsi
    rsi_int = 14
    df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()

# kd

    df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], 17, 5)
    df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], 17, 5)
    
# kd cross
    df['kd_cross'] = check_cross(df, kd_dir)

In [11]:
def conditions(df):

    # c1
    df['c1'] = df['kd_cross']
    # c2
    df['c2'] = df['Close'] >= df['ema_8']
    # c3
    df['c3'] = df['ema_8'] >= df['ema_18']
    # c4
    df['c4'] = df['ema_18'] >= df['ema_38']
    df['c5'] = df['Low'] <= df['ema_38']
    
    # signal
    df['signal'] = df['c1'] & df['c2'] & df['c3'] & df['c4'] & df['c5']
    
    # open_entry
    df['open_entry'] = df['signal'].shift()

In [12]:
def entries(df, sl_atr, tp_atr, sl_det):

    in_position = False
    stop_loss = np.nan
    take_profit = np.nan
    close_val = df['Close']
    atr_val = df['atr']
    
    df['position'] = ''  # Create an empty column for position

    for index, row in df.iterrows():

        if index == 0:
            continue

        elif row['open_entry']:

            df.at[index, 'entry_p'] = close_val.shift(1).at[index]
            df.at[index, 'stop_loss'] = close_val.shift(1).at[index] - sl_atr * atr_val.shift(1).at[index]
            df.at[index, 'take_profit'] = close_val.shift(1).at[index] + tp_atr * atr_val.shift(1).at[index]
            df.at[index, 'position'] = 'Buy'
            in_position = True
            stop_loss = df.at[index, 'stop_loss']
            take_profit = df.at[index, 'take_profit']


        # 吃筍
        #-----------------------------重要-----------------------------
        # 若用 if 寫，則有可能入場馬上吃筍，若用 elif 則一個 iteration 只會執行一次
        elif in_position and row['Close'] <= stop_loss:
            df.at[index, 'position'] = 'Stop'
            in_position = False
            stop_loss = np.nan
            take_profit = np.nan

        # set take profit
        elif in_position and row['High'] >= take_profit:
            df.at[index, 'position'] = 'Sell'
            in_position = False
            stop_loss = np.nan
            take_profit = np.nan
    

    # 過濾有訊號或事件發生的Ｋ線
    df = df[(df['open_entry']) | (df['position'] != '')]

In [13]:
# 部位回測
def backtest(df):

    df = df.reset_index(drop = True)
    df = df[(df['position'] == 'Buy') |
                  (df['position'] == 'Sell') |
                  (df['position'] == 'Stop')]

    # 一次進場多少單位
    pos_size = 200

    col = ['Open_Time', 'Open', 'Close', 'High', 'Low', 'atr', 'position','entry_p', 'stop_loss', 'take_profit']
    pos = df[col]
    pos = pos.reset_index(drop = True)


    for index, row in pos.iterrows():

        current_pos = 0

        # 進場
        if pos.at[index, 'position'] == 'Buy':
            pos.at[index, 'size'] = pos_size / pos.at[index, 'entry_p']
            pos.exit_p = np.nan

        # 出場
        if pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':

            # 停利：達成條件時收盤價
            if pos.at[index, 'position'] == 'Sell':
                for i in range(index -1, -1, -1):
                    if pos.at[i, 'position'] == 'Buy':
                        pos.at[index, 'exit_p'] = pos.at[i, 'take_profit']
                    break

            # 停損：打到進場停損點（往回跌代，直到最近的'Buy'及其'stop_loss'）
            if pos.at[index, 'position'] == 'Stop':
                pos.at[index, 'exit_p'] = pos.at[index, 'Close']
#                 for i in range(index -1, -1, -1):
#                     if pos.at[i, 'position'] == 'Buy':
#                         pos.at[index, 'exit_p'] = pos.at[i, 'stop_loss']
#                     break

            # 計算每次出場部位大小（每次出場皆清倉）
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Buy':
                    current_pos += pos.at[i, 'size']
                    if i == 0:
                        pos.at[index, 'size'] = -current_pos
                    else:
                        continue
                else:
                    pos.at[index, 'size'] = -current_pos
                    current_pos = 0
                    break


    # 計算部位價值
    for index, row in pos.iterrows():
        if pos.at[index, 'position'] == 'Buy':
            pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'entry_p'], 4)
        elif pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
            pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'exit_p'], 4)


    # 若最後一筆為 Buy，移除該單，迭代驗證
    for index, row in pos.iloc[::-1].iterrows():
        if row['position'] == 'Buy':
            pos = pos.drop(index)
        else:
            break


    # 手續費、滑點、價差
    fee = 0.05 / 100
    amt_abs_sum = pos.amt.abs().sum()
    ttl_fee = amt_abs_sum * fee

    # 損益
    leverage = 10
    ttl_profit = -pos.amt.sum() - ttl_fee

    
    # 計算獲利/虧損次數
    agg_amts = []

    for i in range(len(pos) - 1, -1, -1):

        if pos.loc[i, 'position'] in ['Stop', 'Sell']:

            # look out for the + sign
            total_amt = pos.loc[i, 'amt'] + np.absolute(pos.loc[i, 'amt']) * fee

            # iterate backwards from the current row until reaching another 'Stop' or 'Sell'
            # watch out for the + in total_amt += trading_fee
            j = i - 1
            while j >= 0 and pos.loc[j, 'position'] not in ['Stop', 'Sell']:
                total_amt += pos.loc[j, 'amt']
                trading_fee = np.absolute(pos.loc[j, 'amt']) * fee
                total_amt += trading_fee
                j -= 1

            # add the aggregated amount to the list
            agg_amts.append(total_amt)

    agg_amts.reverse()

    
    # 計算進場最大部位，最大損益
    consec_entry = 0
    position_amt_sum = 0
    max_consec_entry = 0
    max_position = 0
    max_profit = 0
    max_loss = 0

    for index, row in pos.iterrows():

        if row['position'] == 'Buy':

            consec_entry += 1
            position_amt_sum += row['amt']

        elif row['position'] in ['Sell', 'Stop']:

            if consec_entry > max_consec_entry:
                max_consec_entry = consec_entry
                max_position = position_amt_sum

            position_amt_sum += row['amt']

            if -position_amt_sum > max_profit:
                max_profit = -position_amt_sum

            if -position_amt_sum < max_loss:
                max_loss = -position_amt_sum

            consec_entry = 0
            position_amt_sum = 0

        else:
            pass


    # 最大部位
    profit_per = "{:.2f}%".format(ttl_profit / (max_position/leverage) * 100)


    # 勝率
    wins = 0
    loses = 0

    for trade in agg_amts:
        if trade < 0:
            wins += 1
        elif trade > 0:
            loses += 1

    win_rate = "{:.2f}%".format(wins / (wins + loses) * 100)
    
    cumulative_values = []
    cumulative_sum = 0

    for value in agg_amts:
        cumulative_sum -= value
        cumulative_values.append(cumulative_sum)

    max_drawdown = min(cumulative_values)

    # 結果
    result = {'Profit': [round(ttl_profit, 2)],
              'Fee': [round(ttl_fee, 2)],
              'Max_Profit': [round(max_profit, 2)],
              'Max_Loss': [round(max_loss, 2)],
              'Max_Entry': [max_consec_entry],
              'Max_Position': [round(max_position, 2)],
              'Profit_%': [profit_per],
              'Win_Rate': [win_rate],
              'PF_Ratio': [round(ttl_profit/ttl_fee, 2)],
              'Max_Drawdown': [round(max_drawdown/ttl_profit, 2)]}


    result_df = pd.DataFrame(result)

    return result_df, ttl_profit, win_rate, ttl_fee

In [14]:
# iterate using start_time, under same time interval, test different variables
def run_backtest():

    results_df = pd.DataFrame()
    
    symbol_list = []
    interval_list = []
    kd_dir_list = []
    sl_atr_list = []
    tp_atr_list = []
    sl_det_list = []

    i = 0

    loop_start_time = time.time()
    
    parameters = [(symbol, interval, kd_dir, sl_atr, tp_atr, sl_det) 
                  for symbol in symbol_arr
                  for interval in interval_arr
                  for kd_dir in kd_dir_arr
                  for sl_atr in sl_atr_arr
                  for tp_atr in tp_atr_arr
                  for sl_det in sl_det_arr]
    
    for symbol, interval, kd_dir, sl_atr, tp_atr, sl_det in parameters:

        df = get_klines(symbol, interval, start_time_d, end_time)
        df = df.reset_index()

        indicators(df, kd_dir)
        conditions(df)
        entries(df, sl_atr, tp_atr, sl_det)
        
        backtest_df, ttl_profit, win_rate, fee = backtest(df)

        results_df = pd.concat([results_df, backtest_df], ignore_index = True)

        symbol_list.append(symbol)
        interval_list.append(interval)
        kd_dir_list.append(kd_dir)
        sl_atr_list.append(sl_atr)
        tp_atr_list.append(tp_atr)
        sl_det_list.append(sl_det)

        i += 1

        print(f" {i} - {symbol}, {interval}, sl: {sl_atr}, tp: {tp_atr}, {sl_det}, {round(ttl_profit, 2)}, {round(fee, 2)}, {round(ttl_profit/fee, 2)}, {win_rate} ")

    loop_end_time = time.time()

    print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")

    results_df['symbol'] = symbol_list
    results_df['interval'] = interval_list
    results_df['kd_dir'] = kd_dir_list
    results_df['sl_atr'] = sl_atr_list
    results_df['tp_atr'] = tp_atr_list
    results_df['sl_det'] = sl_det_list

    results_df = results_df[['symbol', 'interval', 'kd_dir',
                             'sl_atr', 'tp_atr', 'sl_det',
                             'Profit', 'Fee', 'Max_Profit', 'Max_Loss',
                             'Max_Entry', 'Max_Position', 'Profit_%', 'Win_Rate', 'PF_Ratio', 'Max_Drawdown'
                            ]]
    
    return results_df

In [15]:
results_df = run_backtest()
results_df = results_df.sort_values('Profit', ascending = False)
# print(results_df)

 1 - ethusdt, 1h, sl: 1, tp: 3, Close, 17.03, 21.02, 0.81, 35.90% 
 2 - ethusdt, 1h, sl: 1, tp: 5, Close, 11.78, 21.02, 0.56, 22.37% 
 3 - ethusdt, 1h, sl: 1, tp: 9, Close, -44.38, 20.99, -2.11, 15.07% 
 4 - ethusdt, 1h, sl: 1, tp: 12, Close, -67.74, 20.98, -3.23, 12.68% 
 5 - ethusdt, 1h, sl: 3, tp: 3, Close, -107.48, 20.96, -5.13, 49.25% 
 6 - ethusdt, 1h, sl: 3, tp: 5, Close, -90.76, 20.97, -4.33, 35.94% 
 7 - ethusdt, 1h, sl: 3, tp: 9, Close, -145.13, 20.94, -6.93, 22.41% 
 8 - ethusdt, 1h, sl: 3, tp: 12, Close, -141.49, 20.94, -6.76, 17.86% 
 9 - ethusdt, 1h, sl: 5, tp: 3, Close, -129.73, 20.75, -6.25, 59.68% 
 10 - ethusdt, 1h, sl: 5, tp: 5, Close, -216.35, 20.7, -10.45, 45.61% 
 11 - ethusdt, 1h, sl: 5, tp: 9, Close, -310.23, 20.66, -15.02, 30.00% 
 12 - ethusdt, 1h, sl: 5, tp: 12, Close, -309.16, 20.66, -14.97, 22.92% 
 13 - ethusdt, 1h, sl: 7, tp: 3, Close, -13.39, 20.6, -0.65, 69.09% 
 14 - ethusdt, 1h, sl: 7, tp: 5, Close, -125.0, 20.55, -6.08, 56.25% 
 15 - ethusdt, 1h, sl:

 118 - ftmusdt, 15m, sl: 3, tp: 5, Close, -344.49, 84.07, -4.1, 30.42% 
 119 - ftmusdt, 15m, sl: 3, tp: 9, Close, -548.99, 83.97, -6.54, 17.99% 
 120 - ftmusdt, 15m, sl: 3, tp: 12, Close, -389.0, 84.05, -4.63, 15.22% 
 121 - ftmusdt, 15m, sl: 5, tp: 3, Close, -307.07, 83.89, -3.66, 62.20% 
 122 - ftmusdt, 15m, sl: 5, tp: 5, Close, -284.68, 83.9, -3.39, 46.79% 
 123 - ftmusdt, 15m, sl: 5, tp: 9, Close, -863.7, 83.61, -10.33, 27.37% 
 124 - ftmusdt, 15m, sl: 5, tp: 12, Close, -763.41, 83.66, -9.13, 21.79% 
 125 - ftmusdt, 15m, sl: 7, tp: 3, Close, -328.46, 83.88, -3.92, 67.37% 
 126 - ftmusdt, 15m, sl: 7, tp: 5, Close, -96.1, 83.99, -1.14, 54.19% 
 127 - ftmusdt, 15m, sl: 7, tp: 9, Close, -169.12, 83.96, -2.01, 33.93% 
 128 - ftmusdt, 15m, sl: 7, tp: 12, Close, 45.97, 84.07, 0.55, 27.39% 
 129 - sandusdt, 1h, sl: 1, tp: 3, Close, 67.94, 14.64, 4.64, 40.68% 
 130 - sandusdt, 1h, sl: 1, tp: 5, Close, 134.86, 14.67, 9.19, 32.14% 
 131 - sandusdt, 1h, sl: 1, tp: 9, Close, 64.82, 14.44, 4.49,

 232 - woousdt, 15m, sl: 3, tp: 12, Close, -337.86, 70.87, -4.77, 18.32% 
 233 - woousdt, 15m, sl: 5, tp: 3, Close, -347.07, 70.86, -4.9, 62.91% 
 234 - woousdt, 15m, sl: 5, tp: 5, Close, -480.57, 70.8, -6.79, 49.18% 
 235 - woousdt, 15m, sl: 5, tp: 9, Close, -480.93, 70.79, -6.79, 32.48% 
 236 - woousdt, 15m, sl: 5, tp: 12, Close, -649.8, 70.71, -9.19, 25.50% 
 237 - woousdt, 15m, sl: 7, tp: 3, Close, -357.02, 70.86, -5.04, 71.64% 
 238 - woousdt, 15m, sl: 7, tp: 5, Close, -608.17, 70.73, -8.6, 58.08% 
 239 - woousdt, 15m, sl: 7, tp: 9, Close, -471.09, 70.8, -6.65, 42.54% 
 240 - woousdt, 15m, sl: 7, tp: 12, Close, -564.15, 70.75, -7.97, 34.13% 
 241 - woousdt, 15m, sl: 1, tp: 3, Close, -172.17, 67.15, -2.56, 29.75% 
 242 - woousdt, 15m, sl: 1, tp: 5, Close, -155.05, 67.16, -2.31, 20.07% 
 243 - woousdt, 15m, sl: 1, tp: 9, Close, -96.76, 67.19, -1.44, 12.79% 
 244 - woousdt, 15m, sl: 1, tp: 12, Close, -183.95, 67.14, -2.74, 11.55% 
 245 - woousdt, 15m, sl: 3, tp: 3, Close, -184.82, 67

In [16]:
results_df.to_csv(f'results_df_007.csv')